### Labels 숫자로 변경한 이후


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import os
from tqdm import tqdm
tqdm.pandas()
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터셋 전처리(after labels changed #)


## labels 숫자가 가지는 감정

In [3]:
# label_mapping = {
#     '중립': 0,
#     '불안': 1,
#     '분노': 2,
#     '놀람': 3,
#     '상처': 4,
#     '당황': 5,
#     '기쁨': 6,
#     '행복': 7,
#     '혐오': 8,
#     '공포': 9,
#     '슬픔': 10,
# }

In [4]:
#labels 숫자로 변경한 데이터 불러오기
path = '/content/drive/MyDrive/2차프로젝트/data/combined_data_num_kp.csv'
# path = '/content/drive/MyDrive/Colab Notebooks/model/combined_data_num_kp.csv'

In [5]:
df = pd.read_csv(path)

In [6]:
df.head()

,labels,sentence
0,2,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!
1,8,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.
2,0,손님 왔어요.
3,0,손님? 누구?
4,0,몰라요. 팀장님 친구래요.


In [7]:
# 컬럼별로 feature, label 나누기
sentences = df['sentence']
labels = df['labels']

In [8]:
# 데이터 확인
# 데이터를 undersampling 하면 데이터의 개수가 많이 줄어들어서 undersampling 진행은 하지 않았음
df['labels'].value_counts()

0     48623
2     19710
10    17367
3     10764
1     10433
4     10150
5      9804
6      7339
7      7067
8      5649
9      5566
Name: labels, dtype: int64

In [9]:
# 문장과 감정분류 상태가 맞는지 확인
print(df['sentence'][1111])
print(df['labels'][1111])

속상해서 그러잖아요.
10


In [10]:
# 데이터 train_valid_test로 나누기

from sklearn.model_selection import train_test_split
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)
train_sentences, valid_sentences, train_labels, valid_labels = train_test_split(train_sentences, train_labels, test_size=0.1, random_state=42)

In [11]:
# 나눈 데이터 확인
print('sentence 개수')
print('train_sentences:', len(train_sentences))
print('valid_sentences:',len(valid_sentences))
print('test_sentences:',len(test_sentences))
print('labels 개수')
print('train_labels:',len(train_labels))
print('valid_labels:',len(valid_labels))
print('test_labels:',len(test_labels))

sentence 개수
train_sentences: 109779
valid_sentences: 12198
test_sentences: 30495
labels 개수
train_labels: 109779
valid_labels: 12198
test_labels: 30495


In [12]:
# 데이터 타입 확인
print(type(train_sentences))
print('*' * 50)
print(type(train_labels))

<class 'pandas.core.series.Series'>
**************************************************
<class 'pandas.core.series.Series'>


In [13]:
# 데이터 타입 리스트로 변환
train_labels = train_labels.tolist()
valid_labels = valid_labels.tolist()
test_labels = test_labels.tolist()

In [14]:
# 데이터 타입 확인
print(type(train_sentences))
print('*' * 50)
print(type(train_labels))

<class 'pandas.core.series.Series'>
**************************************************
<class 'list'>


In [15]:
# Bert 형식으로 만들기 위해 문장에 CLS, SEP 추가하기
train_sentences = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in train_sentences]
valid_sentences = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in valid_sentences]
test_sentences = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in test_sentences]

In [16]:
# 바뀐 문장 확인하기
print(test_sentences[:5])
print('*' * 100)
print(test_labels[:5])

['[CLS] 화학약품이나 인위적인 조작 없이 산책이나 체조, 명상 등을 통해서 치료를 하게 됩니다. [SEP]', '[CLS] 오늘도 폐지 주워 오천 원밖에 못 벌었어. 앞으로 어떻게 살지 왜 난 이따위 사람일까. [SEP]', '[CLS] 아내랑 다퉜는데 내가 너무 부정적으로 대처한 것 같아서 마음이 안 좋아. [SEP]', '[CLS] 우리나라의 문제점은 잘알고있으나 지네나라의 문제점은 잘모르는군 [SEP]', '[CLS] 주모 있는거 다 가져와 [SEP]']
****************************************************************************************************
[0, 2, 10, 2, 0]


In [17]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

In [18]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00


### BERT tokenizer를 이용한 전처리

In [19]:
# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [20]:
# 한국어 BERT 중 하나인 'klue/bert-base를 사용
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [21]:
# 데이터 텐서로 변환
MAX_LEN = 128

def data_to_tensor(sentences, labels):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating='post', padding='post')

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return  tensor_inputs, tensor_labels, tensor_masks

In [22]:
# 데이터 텐서로 변환
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
valid_inputs, valid_labels, valid_masks = data_to_tensor(valid_sentences, valid_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [23]:
print(tokenizer.decode([0]))
print(tokenizer.decode([1]))
print(tokenizer.decode([2]))
print(tokenizer.decode([3]))

[PAD]
[UNK]
[CLS]
[SEP]


In [24]:
print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])

tensor([    2,  5021,  1889,  2015,  2259,  1912,    18,  3820,  2259,  1507,
         2073,   617,  2231,  3806,  1521,  2205,  2180,  2778,    35,  3866,
         5079,  5678,  2119,  1527,  2088, 11624,  2052,  1415,  2051,    18,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [25]:
# 배치크기 = 32 파이토치 데이터로더(배치 단위로 데이터를 꺼내올 수 있돌록 하는 모듈로 변환)
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [26]:
print('train_data:', len(train_labels))
print('valid_data:', len(valid_labels))
print('test_data:', len(test_labels))

train_data: 109779
valid_data: 12198
test_data: 30495


In [27]:
# GPU setting
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
    print('cpu')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [28]:
# Torch 버전 확인
print(torch.__version__)
# CUDA 사용 가능여부(True, False)
print(torch.cuda.is_available())
# CUDA 버전
print(torch.version.cuda)
# GPU 사용가능 개수
print(torch.cuda.device_count())
# GPU 이름
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())
torch.backends.cudnn.deterministic = True

2.0.1+cu118
True
11.8
1
Tesla T4
0


### 모델 로드

In [29]:
num_labels = 11

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [30]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
import torch
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

# 모델, 옵티마이저, 스케줄러 정의 (이미 정의되어 있다고 가정)
epochs=5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# 체크포인트 저장 및 불러오기
def save_checkpoint(epoch, model, optimizer, scheduler, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }
    torch.save(checkpoint, filename)

def load_checkpoint(model, optimizer, scheduler, filename):
    checkpoint = torch.load(filename)
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return epoch

# 이어서 학습할 경우
# start_epoch = 0  # 처음부터 학습을 시작하는 경우
start_epoch = load_checkpoint(model, optimizer, scheduler, '/content/drive/MyDrive/2차프로젝트/model/Bert_checkpoint_1.pth')  # 체크포인트에서 이어서 학습하는 경우

epochs = 2  # 이어서 학습할 총 에폭 수
for epoch in range(start_epoch, epochs):
    # 학습 코드 작성

    # 체크포인트 저장
    save_checkpoint(epoch, model, optimizer, scheduler, '/content/drive/MyDrive/2차프로젝트/model/Bert_checkpoint_0815.pth')

In [32]:
# epochs=2
# total_steps = len(train_dataloader) * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [33]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [34]:
def metrics(predictions, labels):
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}
    return metrics

### 모델학습

In [39]:
# 랜덤 시드값.
seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 중간 및 최종 결과 저장 경로 설정
path = "/content/drive/MyDrive/2차프로젝트/model"

model.zero_grad()
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # 모델 저장 (학습 중간 결과 저장)
    # torch.save(model.state_dict(), path + 'Bert_11_emotions_model_epoch_{:04d}_0815.pt'.format(epoch_i + 1))

# 모델 저장 (최종 결과 저장)
torch.save(model.state_dict(), path + 'Bert_11_emotions_model_final_0816.pt')

======== Epoch 1 / 2 ========


500it [05:10,  1.62it/s]

  Batch   500  of  3,431.    Elapsed: 0:05:11.


1000it [10:19,  1.62it/s]

  Batch 1,000  of  3,431.    Elapsed: 0:10:19.


...

  Batch 2,500  of  3,431.    Elapsed: 0:25:44.


3000it [30:52,  1.62it/s]

  Batch 3,000  of  3,431.    Elapsed: 0:30:52.


3431it [35:17,  1.62it/s]



  Average training loss: 0.9179
  Training epcoh took: 0:35:18


In [38]:
# 모델 저장 (최종 결과 저장)
torch.save(model.state_dict(), path + 'Bert_11_emotions_model_final_0816.pt')

RuntimeError: ignored

# 검증 데이터에 대한 평가

In [42]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in label_ids:
        accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

Accuracy: 0.6483
F1 (Macro) Score: 0.5862
F1 (Micro) Score: 0.6483
F1 (Weighted) Score: 0.6418


In [ ]:
# 모델 저장
torch.save(model.state_dict(), path + 'Bert_11_emotions_model_3.pt')

In [45]:
# 모델 로드
# model.load_state_dict(torch.load("/content/drive/MyDrive/Bert_11_emotions_model_kp.pt"))
model.load_state_dict(torch.load('/content/drive/MyDrive/2차프로젝트/model/Bert_11_emotions_model_kp.pt'))

<All keys matched successfully>

### 테스트 데이터 평가

In [46]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in label_ids:
        accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

100it [00:21,  4.43it/s]

  Batch   100  of    953.    Elapsed: 0:00:22.


200it [00:44,  4.63it/s]

  Batch   200  of    953.    Elapsed: 0:00:44.


300it [01:05,  4.74it/s]

...

  Batch   800  of    953.    Elapsed: 0:02:52.


900it [03:13,  4.70it/s]

  Batch   900  of    953.    Elapsed: 0:03:13.


953it [03:24,  4.66it/s]

Accuracy: 0.7460
F1 (Macro) Score: 0.7089
F1 (Micro) Score: 0.7460
F1 (Weighted) Score: 0.7438


### 예측

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification",
                model=model.cuda(),
                tokenizer=tokenizer,
                device=0,
                max_length=512,
                return_all_scores=True,
                function_to_apply='softmax')

In [ ]:
result = pipe('오늘 프로젝트가 끝날 것 같아서 기분이 좋다')
print(result)

In [ ]:
pipe = pipeline("text-classification",
                model=model.cuda(),
                tokenizer=tokenizer, device=0,
                max_length=512,
                function_to_apply='softmax')

In [ ]:
# label_dict를 통해 숫자로 분류되어진 라벨값을 다시 감정으로 변환
label_dict = {'LABEL_0': '중립',
              'LABEL_1': '불안',
              'LABEL_2': '분노',
              'LABEL_3': '놀람',
              'LABEL_4': '상처',
              'LABEL_5': '당황',
              'LABEL_6': '기쁨',
              'LABEL_7': '행복',
              'LABEL_8': '혐오',
              'LABEL_9': '공포',
              'LABEL_10': '슬픔',
              }

In [ ]:
# 예측
def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]

In [ ]:
def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]

In [ ]:
prediction('오늘 프로젝트가 끝날 것 같아서 기분이 좋다')

In [ ]:
prediction('비가 오니 슬픈 예감이 드는 이유는 왜일까')

### 입력한 감정 예측

In [ ]:
prediction(input('당신의 기분 또는 상태를 입력해주세요: '))